На основе уже знакомого тебе тестового набора данных из первого задания посчитай следующие метрики в тетрадке Jupyter Notebook:
* Delivery rate
* Open rate
* Click to Open rate
* Unsubscribe rate
* Выяви лучшую тему
* И лучший день недели (для планирования рассылок)


In [1]:
# импортируем библиотеки
import pandas as pd
import datetime as dt

import numpy as np
import matplotlib.pyplot as plt

from io import BytesIO
import requests
import re

import warnings
warnings.simplefilter('ignore')

In [2]:
# задаем настройки отображения
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_colwidth = 150 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 200)

In [3]:
# функция, забирающая данные из гугл-таблиц
def download(spreadsheet_id):
    url = 'https://docs.google.com/spreadsheets/d/{}/export?format=xlsx'.format(spreadsheet_id)
    r = requests.get(url)
    return pd.read_excel(BytesIO(r.content))

In [4]:
# присваиваем переменной загруженную таблицу
data = download("1NVEr062eowesdXxZ7OkaRdjC7r-QB8b62tv1j2cs2cs")
data.head()

,Название рассылки,Название кампании,Направление,Месяц,Дата,Год,Номер недели,День недели,День недели.1,Время,Веб-версия,Тема письма,Сегмент,Отправлено,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,UTM Метка,Пользователей на сайте,Воронка продаж. Шаг 1,Воронка продаж. Шаг 2,Воронка продаж. Шаг 3
0,Название рассылки 1,Название кампании 1,Email,Октябрь,2021-10-27,2021,43,3,03-среда,19:24:00,#ССЫЛКА,Тема письма 1,Сегмент 1,"780,789.00","741,749.55","148,349.91","17,801.99","39,039.45","7,417.50",Метка 1,"16,377.83","6,336.58","6,209.85","5,154.18"
1,Название рассылки 10,Название кампании 10,Email,Ноябрь,2021-11-05,2021,45,5,05-пятница,12:02:00,#ССЫЛКА,Тема письма 10,Сегмент 2,"719,370.00","683,401.50","123,012.27","11,071.10","35,968.50","6,834.02",Метка 10,"10,296.13","3,558.34","3,095.76","2,538.52"
2,Название рассылки 100,Название кампании 100,Email,Апрель,2022-04-11,2022,15,1,01-понедельник,16:26:00,#ССЫЛКА,Тема письма 100,Сегмент 3,"1,201,415.00","1,141,344.25","182,615.08","15,339.67","60,070.75","11,413.44",Метка 100,"13,959.10","3,369.73","2,864.27","2,262.77"
3,Название рассылки 101,Название кампании 101,Email,Апрель,2022-04-12,2022,15,2,02-вторник,16:26:00,#ССЫЛКА,Тема письма 101,Сегмент 1,"1,393,827.00","1,324,135.65","264,827.13","10,328.26","69,691.35","13,241.36",Метка 101,"8,985.58","5,116.39","4,277.30","3,207.98"
4,Название рассылки 102,Название кампании 102,Email,Апрель,2022-04-13,2022,15,3,03-среда,16:26:00,#ССЫЛКА,Тема письма 102,Сегмент 2,"1,276,821.00","1,212,979.95","218,336.39","15,720.22","63,841.05","12,129.80",Метка 102,"10,846.95","2,816.95","2,287.37","1,669.78"


In [5]:
# фунция для получения общей информации о датасетах
def get_info(df, df_name):
    print(f'Общая информация о таблице - {df_name}')
    print(df.info() , '\n') # общую информацию
    print('*'*50)
    print(f'Количество NaN значений в таблице - {df_name}', '\n') 
    print(df.isna().sum()) # количество пропусков
    print('*'*50)
    print(f'Число дубликатов в таблице - {df_name}', '\n') 
    print(df.duplicated().sum()) # количество дубликатов

In [6]:
# вызываем функцию
get_info(data, 'data')

Общая информация о таблице - data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Название рассылки       218 non-null    object        
 1   Название кампании       218 non-null    object        
 2   Направление             218 non-null    object        
 3   Месяц                   218 non-null    object        
 4   Дата                    218 non-null    datetime64[ns]
 5   Год                     218 non-null    int64         
 6   Номер недели            218 non-null    int64         
 7   День недели             218 non-null    int64         
 8   День недели.1           218 non-null    object        
 9   Время                   218 non-null    object        
 10  Веб-версия              218 non-null    object        
 11  Тема письма             218 non-null    object        
 12  Сегмент         

* 218 строк
* Числовые данные с "Отправлено" по "Воронка продаж" тип float, в исходной таблице отображен int.
* NaN и дубликаты отсутствуют

In [7]:
# Выбираем столбцы с числами с плавающей точкой
float_cols = data.select_dtypes(include=['float64']).columns
# Преобразуем эти столбцы к целочисленному типу
data[float_cols] = data[float_cols].applymap(np.int64)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Название рассылки       218 non-null    object        
 1   Название кампании       218 non-null    object        
 2   Направление             218 non-null    object        
 3   Месяц                   218 non-null    object        
 4   Дата                    218 non-null    datetime64[ns]
 5   Год                     218 non-null    int64         
 6   Номер недели            218 non-null    int64         
 7   День недели             218 non-null    int64         
 8   День недели.1           218 non-null    object        
 9   Время                   218 non-null    object        
 10  Веб-версия              218 non-null    object        
 11  Тема письма             218 non-null    object        
 12  Сегмент                 218 non-null    object    

In [8]:
# удалим воронку
data.drop(data.columns [[19, 20, 21, 22, 23]], axis= 1 , inplace= True ) 

**Расчет основных метрик**

In [9]:
# Delivery rate
data['delivery_rate'] = data['Доставлено']/data['Отправлено']*100
data['delivery_rate'].describe()

count   218.00
mean     97.65
std       1.08
min      95.00
25%      98.00
50%      98.00
75%      98.50
max      98.50
Name: delivery_rate, dtype: float64

Средняя доставляемость 97,65%. По данным unisender.com: "нормальным значением delivery rate считают показатель равный 95% и выше". 

In [10]:
# Open rate
data['open_rate'] = data['Открытия']/data['Доставлено']*100
data['open_rate'].describe()

count   218.00
mean     13.76
std       3.39
min       9.20
25%      11.05
50%      13.99
75%      17.43
max      20.00
Name: open_rate, dtype: float64

Средняя открываемость 13.76%. По данным unisender.com: "Нормального или среднего open rate не существует. Результаты зависят от сферы бизнеса, качества контента, аудитории, сезонности и десятка других показателей. Согласно исследованию Mindbox, максимальная медиана OR (число, которое находится в середине набора чисел, если его упорядочить по возрастанию) в 2022 году была зафиксирована в сфере недвижимости (36%), а минимальная — в сфере ювелирных товаров (11%)".

In [11]:
# Click to Open rate
data['ctor'] = data['Клики']/data['Открытия']*100
data['ctor'].describe()

count   218.00
mean      8.12
std       2.63
min       3.90
25%       7.20
50%       8.40
75%       9.00
max      12.00
Name: ctor, dtype: float64

CTOR в среднем 8,12%, колеблется от 3,9% до 12%. Хороший CTOR, по данным сервиса email-рассылок SendGrid, — от 20 до 30%.

In [12]:
# Unsubscribe rate
data['ur'] = data['Отписки']/data['Отправлено']*100
data['ur'].describe()

count   218.00
mean      2.42
std       4.10
min       0.15
25%       0.56
50%       0.62
75%       0.89
max      12.80
Name: ur, dtype: float64

UR (Unlikes Rate) = (Количество отписавшихся / Общее количество подписчиков) * 100%
По данным data не очевидно общее число подписчиков. Вероятно, оно зависит от сегмента и иных параметров. Примем за общее число подписчиков рассылки число отправленных писем.

В среднем 2,42%. Максимальное значение в 5 раз больше среднего. Некоторые темы, очевидно, были неудачными.

**Выявляем лучшую тему**

In [13]:
# посмотрим на объем рассылок
data[['Отправлено']].describe()

,Отправлено
count,218.00
mean,"1,476,655.67"
std,"587,458.73"
min,"510,034.00"
25%,"930,482.50"
50%,"1,497,056.00"
75%,"1,988,811.00"
max,"2,492,076.00"


Самая большая рассылка почти в 5 раз больше самой маленькой 

In [14]:
#рассылки, после которых отписалось человек, больше, чем среднее значение отписок
ur_avg = 2.42 # среднее значение отписок по всем рассылккам
u_mail = data.loc[(data['ur'] > ur_avg) ]
u_mail.describe()

,Год,Номер недели,День недели,Отправлено,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,delivery_rate,open_rate,ctor,ur
count,36.00,36.00,36.00,36.00,36.00,36.00,36.00,36.00,36.00,36.00,36.00,36.00,36.00
mean,"2,021.33",33.06,3.19,"1,596,791.11","1,567,121.33","206,202.75","17,201.67","29,668.83","184,210.03",98.17,13.21,8.21,11.56
std,0.48,22.25,1.72,"570,987.25","559,430.74","96,850.58","10,692.55","12,112.11","67,852.15",0.24,3.68,2.71,0.97
min,"2,021.00",1.00,1.00,"560,494.00","549,284.00","82,863.00","3,231.00","11,209.00","71,406.00",98.00,9.20,3.90,10.78
25%,"2,021.00",3.00,1.00,"1,084,843.75","1,065,820.50","130,312.25","8,760.00","18,016.25","117,600.75",98.00,11.05,7.20,10.78
50%,"2,021.00",46.00,4.00,"1,638,312.50","1,605,546.00","182,148.50","14,888.50","28,864.50","182,183.50",98.00,11.05,8.40,10.83
75%,"2,022.00",50.00,5.00,"2,156,240.25","2,113,115.00","253,297.00","21,777.00","42,292.25","236,255.00",98.50,17.43,9.00,12.74
max,"2,022.00",52.00,5.00,"2,434,236.00","2,385,551.00","390,973.00","46,527.00","48,684.00","303,771.00",98.50,18.00,12.00,12.80


В рассылки с отписками попали 36 строк, это 16.51% всех рассылок. Объем рассылок различается те же приблизительно, в 5 раз. Нельзя сказать, что количество отписок связано только с их количеством.

In [15]:
# проверим корреляцию
corr = data[['Отправлено', 'Отписки']].corr().round(2)
corr.style.background_gradient(cmap='coolwarm')

,Отправлено,Отписки
Отправлено,1.000000,0.270000
Отписки,0.270000,1.000000


Для трактовки силы связи между двумя наблюдаемыми величинами используется шкала Чеддока. Она определяет тесноту связи следующим образом:

* 0 – полное отсутствие связи;
* 0 – 0.3 – очень слабая;
* 0.3 – 0.5 – слабая;
* 0.5 – 0.7 – средняя;
* 0.7 – 0.9 – высокая;
* 0.9 – 1 – очень высокая.
* 1 – абсолютная взаимозависимость.

В данном случае корреляция 0,27 - очень слабая. Действительно нельзя сказать, что количество отписок связано только с количеством писем в рассылке.

In [16]:
#рассылки, после которых отписалось человек, меньше, чем 75% отписок
ur_sigm = 0.89 # среднее значение отписок по всем рассылккам
sigm_mail = data.loc[(data['ur'] < ur_sigm) ]
sigm_mail.describe()

,Год,Номер недели,День недели,Отправлено,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,delivery_rate,open_rate,ctor,ur
count,177.00,177.00,177.00,177.00,177.00,177.00,177.00,177.00,177.00,177.00,177.00,177.00,177.00
mean,"2,021.40",26.79,3.51,"1,463,583.25","1,429,305.77","194,752.87","15,705.41","34,276.49","8,806.61",97.62,13.74,8.11,0.60
std,0.49,17.37,1.39,"592,315.26","579,774.19","91,925.72","9,154.03","20,433.41","5,036.33",1.08,3.28,2.62,0.23
min,"2,021.00",1.00,1.00,"510,034.00","499,833.00","50,529.00","1,970.00","8,136.00","1,036.00",95.00,9.20,3.90,0.15
25%,"2,021.00",12.00,3.00,"925,705.00","907,637.00","127,471.00","9,826.00","18,417.00","4,754.00",97.00,11.05,7.20,0.55
50%,"2,021.00",26.00,4.00,"1,502,112.00","1,457,691.00","176,109.00","13,269.00","31,744.00","7,953.00",98.00,14.00,8.40,0.59
75%,"2,022.00",45.00,5.00,"1,956,368.00","1,917,240.00","259,110.00","19,750.00","42,147.00","11,785.00",98.00,17.43,9.00,0.88
max,"2,022.00",52.00,6.00,"2,492,076.00","2,441,118.00","436,664.00","47,292.00","118,932.00","21,833.00",98.50,18.00,12.00,0.89


Осталось 177 строк, чуть более 81% от общего числа рассылок.

In [17]:
#отфильтруем рассылки, которые лучше открывают
or_sigm = 17.43
sigm_mail = sigm_mail.loc[(sigm_mail['open_rate'] > or_sigm) ]
sigm_mail.describe()

,Год,Номер недели,День недели,Отправлено,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,delivery_rate,open_rate,ctor,ur
count,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00,29.00
mean,"2,021.45",26.31,3.24,"1,423,064.93","1,394,283.62","249,006.62","18,514.41","28,780.31","9,191.69",97.97,17.88,7.76,0.62
std,0.51,18.46,1.66,"661,959.26","649,098.90","115,468.27","10,180.73","14,058.46","6,177.06",0.44,0.23,2.50,0.24
min,"2,021.00",2.00,1.00,"542,459.00","534,322.00","96,177.00","5,947.00","8,136.00","1,290.00",97.00,17.43,3.90,0.15
25%,"2,021.00",8.00,2.00,"838,103.00","821,340.00","147,841.00","11,762.00","16,762.00","4,236.00",98.00,18.00,7.20,0.49
50%,"2,021.00",33.00,4.00,"1,470,010.00","1,425,909.00","248,536.00","15,260.00","26,204.00","7,021.00",98.00,18.00,8.40,0.57
75%,"2,022.00",45.00,5.00,"2,054,933.00","2,013,834.00","362,490.00","26,765.00","42,147.00","13,331.00",98.00,18.00,9.00,0.88
max,"2,022.00",52.00,6.00,"2,475,420.00","2,425,911.00","436,664.00","47,262.00","51,320.00","21,833.00",98.50,18.00,12.00,0.89


Осталось 29 рассылок. Среднее число кликликабельности из открытых писем снизилось, максимум, минимум и медиана кликабельности не изменились. 

In [18]:
# отфильтруем рассылки, где активнее кликают
ctor_sigm = 9
sigm_mail = sigm_mail.loc[(sigm_mail['ctor'] > ctor_sigm) ]
sigm_mail.describe()

,Год,Номер недели,День недели,Отправлено,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,delivery_rate,open_rate,ctor,ur
count,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00
mean,"2,021.25",30.00,3.25,"1,353,752.00","1,322,399.75","233,731.50","28,047.25","31,351.25","8,389.25",97.75,17.71,12.00,0.65
std,0.50,13.64,0.96,"796,116.71","777,680.45","135,926.21","16,311.26","20,201.56","4,149.24",0.50,0.33,0.00,0.15
min,"2,021.00",10.00,2.00,"624,983.00","612,483.00","110,247.00","13,229.00","12,499.00","3,613.00",97.00,17.43,12.00,0.57
25%,"2,021.00",27.25,2.75,"736,438.25","721,709.25","126,666.75","15,199.25","14,728.00","6,020.50",97.75,17.43,12.00,0.58
50%,"2,021.00",35.00,3.50,"1,242,132.50","1,208,736.00","215,411.50","25,849.00","30,793.00","8,306.50",98.00,17.71,12.00,0.58
75%,"2,021.25",37.75,4.00,"1,859,446.25","1,809,426.50","322,476.25","38,697.00","47,416.25","10,675.25",98.00,18.00,12.00,0.65
max,"2,022.00",40.00,4.00,"2,305,760.00","2,259,644.00","393,856.00","47,262.00","51,320.00","13,331.00",98.00,18.00,12.00,0.88


Остались 4 рассылки, с максимальным ctor

In [19]:
sigm_mail

,Название рассылки,Название кампании,Направление,Месяц,Дата,Год,Номер недели,День недели,День недели.1,Время,Веб-версия,Тема письма,Сегмент,Отправлено,Доставлено,Открытия,Клики,Баунсы (Все ошибки),Отписки,delivery_rate,open_rate,ctor,ur
50,Название рассылки 144,Название кампании 144,Email,Август,2021-08-17,2021,33,2,02-вторник,12:04:00,#ССЫЛКА,Тема письма 144,Сегмент 2,2305760,2259644,393856,47262,46115,13331,98.00,17.43,12.00,0.58
55,Название рассылки 149,Название кампании 149,Email,Сентябрь,2021-09-16,2021,37,4,04-четверг,12:04:00,#ССЫЛКА,Тема письма 149,Сегмент 1,624983,612483,110247,13229,12499,3613,98.00,18.00,12.00,0.58
65,Название рассылки 158,Название кампании 158,Email,Октябрь,2021-10-06,2021,40,3,03-среда,12:04:00,#ССЫЛКА,Тема письма 158,Сегмент 3,1710675,1659354,298683,35842,51320,9790,97.00,18.00,12.00,0.57
200,Название рассылки 83,Название кампании 83,Email,Март,2022-03-10,2022,10,4,04-четверг,14:33:00,#ССЫЛКА,Тема письма 83,Сегмент 1,773590,758118,132140,15856,15471,6823,98.00,17.43,12.00,0.88


Основываясь на данных расчитанных выше можно казать, что лучшая тема для рассылки 158. При максимальной открываемости и равной кликабельности у нее самый низкий рейтинг отписок из оставшихся тем. 

**Лучший день недели (для планирования рассылок)**

In [20]:
# рассылки по дням недели
data['День недели'].value_counts().sort_values()

6     5
2    25
1    34
3    40
4    54
5    60
Name: День недели, dtype: int64

Больше всего рассылок отправлено в пятницу - 60 и четверг - 54, меньше всего в субботу - 5, в воскресенье рассылки не проводились.

In [21]:
# число писем по дням недели
data.groupby('День недели')\
  .agg({'Отправлено':['median','mean','min','max']})\
  .style.format("{:.2f}")

Меньше всего писем было в субботних рассылках, больше всего во вторник. 

In [22]:
# как отписывались по дням недели
data.groupby('День недели')\
  .agg({'ur':['median','mean','min','max']})\
  .style.format("{:.2f}")

В среднем чаще всего отписываются от рассылок в пятницу и понедельник, реже в субботу и среду. По вторникам самое высокое минимальное значение отписок.

In [23]:
# в какие дни письма открывали
data.groupby('День недели')\
  .agg({'open_rate':['median','mean','min','max']})\
  .style.format("{:.2f}")

Минимум одинаковый во все дни. Максимальная открываемость во вторник и среду. В среднем по окрываемости среда лидирует. В медиане среда и суббота.

In [24]:
# кликабельность по дням недели
data.groupby('День недели')\
  .agg({'ctor':['median','mean','min','max']})\
  .style.format("{:.2f}")

В среднем и медианном значении лучше кликали по ссылкам в открытых письмах во вторник. Меньше всего в субботу.

Исходя из показателей кликабельности и отписок лучший день - среда. Перспективный день - суббота. 